# Urban dashboard

## Listes villes 


In [1]:
#Get dependencies 

import pickle
from string import Template
import geocoder

from string import Template
import json
from urllib.request import urlopen
import urllib

import re
import pickle


### Phase 1: From a city list to var City and var Country

In [8]:

# To be set in a json
Maroc = ["Fès", "Rabat", "Oujda", "Agadir", "Kénitra", "Beni melal","Marrakech","Khourigba","Ej Jadida","Settat","Safi","Khenisset","Laayoune","Meknes","Tanger"]
Benin = ["Dassa-Zoumé", "Cotonou", "Sémé-Podji"]
Burkina=["Bobo-Dioulasso"]
Cameroun = ["Yaoundé", "Douala", "Bamenda", "Bafoussam", "Ngaoundéré", "Nkongsamba", "Belel", "Garoua", "Maroua"]
Congo = ["Brazzaville"]
Ivory_coast = ["Treichville", "Yamoussoukro", "Bocanda"]
Gabon = ["Libreville"]
indonesie = ["Jakarta", "Surabaya", "Bandung"]
Madagascar = ["Fianarantsoa"]
Togo = ["Tsévié","Lomé"]
Tunisie = ["Beni Khiar"]
rep_dom = ["Santo Domingo"] 
Ghana = ["Accra"]
liste_pays = {"Gabon": Gabon, "Madagascar": Madagascar, "Togo":Togo, "Ghana":Ghana, "Bénin": Benin, "Burkina Faso": Burkina,"Congo Brazza": Congo, "Côte d'ivoire":Ivory_coast, "Maroc": Maroc, "Cameroun": Cameroun, "Indonésie": indonesie, "République Dominicaine": rep_dom, "Tunisie": Tunisie}



country = 1
menu =""
dict_country = ""
dict_menu= ""
dicte = ""
dict_gps = ""
coo = {}
bbox = {}


for i in liste_pays:

    city = 1
    Input_p = {"id": str(country), "i": i }
    
    menu += Template('{id: "$id", name:"$i" },').safe_substitute(Input_p)
    dict_country += Template('"$id":"$i",').safe_substitute(Input_p)
    dict_menu_t= ''
    
    for j in liste_pays[i]:
        g = geocoder.google(j)
        
        t =g.latlng
        k= str(country)+"_"+str(city)
        
        Input_c = {"k":k, "i": j, "gps": t}
        
        
        
        dict_menu_t += Template('{id: "$k", name:"$i" },').safe_substitute(Input_c)
        dicte +=  Template('"$k":"$i",').safe_substitute(Input_c)
        dict_gps +=  Template('"$k":$gps,').safe_substitute(Input_c)

        coo[j] =  t
        bbox[j] = [t[0]-0.15, t[1]-0.15, t[0]+0.15, t[1]+0.15]
        
        city +=1 
        
        
    dict_menu += '"'+i+'":['+dict_menu_t+'],'
    country +=1
    
    selected_country = '"'+i+'"'
    selected_city = '"' + j + '"'
    selected_gps = t


dict_menu = "{"+dict_menu[:-1]+"}"
dict_gps = "{"+dict_gps[:-1]+"}"
dicte = "{"+dicte[:-1]+"}"
menu= "["+menu+"]"
dict_country = "{"+dict_country[:-1]+'}'


# To be saved in a db ?
with open('cities.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([coo, bbox, dict_menu, dict_gps, dicte, menu , dict_country,  selected_country, selected_city, selected_gps], f)



### Phase 2: Do OSM request and create corresponding JSON


In [3]:

# Getting back the objects:
with open('cities.pickle','rb') as f:  # Python 3: open(..., 'rb')
    coo, bbox, dict_menu, dict_gps, dicte, menu , dict_country,   selected_country, selected_city, selected_gps = pickle.load(f)

    
    
http = 'http://overpass-api.de/api/interpreter?'


dict_query = {'hospital' : '''[out:json];
      node["amenity"="hospital"]$bbox; out geom; way["amenity"="hospital"]$bbox;
      out geom center qt; relation["amenity"="hospital"]$bbox; out geom center qt; way["amenity"="hospital"]$bbox;
      out body qt; relation["amenity"="hospital"]$bbox; out body qt;
      ''' , 'uni' : '''[out:json];
      node["amenity"="university"]$bbox;
      out geom;
      way["amenity"="university"]$bbox;
      out geom center qt;
      relation["amenity"="university"]$bbox;
      out geom center qt;      way["amenity"="university"]$bbox;
      out body  qt;
      relation["amenity"="university"]$bbox;
      out body  qt; 
      ''' , 'townhall' : ''' [out:json];
      node["amenity"="townhall"]$bbox;
      out body;
      way["amenity"="townhall"]$bbox;
      out body center qt;
      relation["amenity"="townhall"]$bbox;
      out body center qt;      way["amenity"="townhall"]$bbox;
      out body qt;
      relation["amenity"="townhall"]$bbox;
      out body  qt;
    ''' , 'marketplace' :''' [out:json];
    node["amenity"="marketplace"]$bbox;
    out body;
      way["amenity"="marketplace"]$bbox;
      out body center qt;
      relation["amenity"="marketplace"]$bbox;
      out body center qt;    way["amenity"="marketplace"]$bbox;
      out body  qt;
      relation["amenity"="marketplace"]$bbox;
      out body qt;
    
    ''', 'bus':''' [out:json];
    node["amenity"="bus_station"]$bbox;
    out body;
      way["amenity"="bus_station"]$bbox;
      out body center qt;
      relation["amenity"="bus_station"]$bbox;
      out body center qt;    way["amenity"="bus_station"]$bbox;
      out body  qt;
      relation["amenity"="bus_station"]$bbox;
      out body qt;
    
    ''' }

for i in bbox:
    temp = str(bbox[i])
    bb = "("+temp[1:-1]+")"
    Input = {'bbox': bb}
    for z in dict_query:
        query = Template(dict_query[z]).safe_substitute(Input)
        
        print(query)
        query_string = urllib.parse.urlencode({'data': query})
        query_string = query_string.encode('utf-8')
        print(type(query_string))

        try:
            response = urlopen(url=http, data=query_string).read()
        except :
            print("Oups")

        Input_save = {'query' : z, 'nam' : i}
        name = Template('$nam-$query-center.json').safe_substitute(Input_save)
        
        with open('data2/'+name, "w") as f: f.write(response.decode('utf-8'))

            
#json still need to be processed with bash + osmtogeojson 
# IN BASH: for file in *.json; do osmtogeojson  "$file" >  "${file%.json}.geojson" ; done

[out:json];
      node["amenity"="hospital"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676); out geom; way["amenity"="hospital"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676);
      out geom center qt; relation["amenity"="hospital"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676); out geom center qt; way["amenity"="hospital"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676);
      out body qt; relation["amenity"="hospital"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676); out body qt;
      
<class 'bytes'>
[out:json];
      node["amenity"="university"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676);
      out geom;
      way["amenity"="university"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676);
      out geom center qt;
      relation["amenity"="university"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676);
      out geom center qt;      way["amenity"="university"](0.26619760000000003, 9.3172676, 0.5661976, 9.6172676);
      out body

### Phase 3:Create var City and var Country 



In [ ]:
with open('cities.pickle') as f:  # Python 3: open(..., 'rb')
    coo, bbox, dict_menu, dict_gps, dicte, menu , dict_country,  selected_country, selected_city, selected_gps = pickle.load(f)

Input = {"menu": menu, "dict_country":dict_country, "selected_country": selected_country, "dict_menu" : dict_menu , "dicte": dicte, "dict_gps": dict_gps, "selected_city":selected_city, "selected_gps":selected_gps}

template_country = '''
var Country = {

mymap: "",
menu: $menu,
dict: $dict_country,
selected: $selected_country
}
'''


template_city = '''var City = {
	mymap: "",

	dict_menu: $dict_menu ,

	dict: $dicte,
	dict_gps: $dict_gps,
	selected: $selected_city,
	selectedgps: $selected_gps,


};'''


city = Template(template_city).safe_substitute(Input)
country = Template(template_country).safe_substitute(Input)

#save in a js file
with open("citydashboard_data.js", "w") as f: f.write(city+" "+country)
